# Parser

Code to take some files in the output format of the URL scraper, and translate them to the input format of the Python ML models

In [2]:
import json

In [3]:
FILES = ["dailymail_lab.json", "guardian.json"]
OUTPUT = "data_to_label_clean.json"

In [4]:
result = {"data": []}
for file in FILES:
    with open(file) as f:
        entries = json.load(f)
        for entry in entries:
            result["data"].append(
                {
                    "text": entry["main_heading"],
                    "agency": False,
                    "humanComparison": False,
                    "hyperbole": False,
                    "historyComparison": False,
                    "unjustClaims": False,
                    "deepSounding": False,
                    "sceptics": False,
                    "deEmphasize": False,
                    "performanceNumber": False,
                    "inscrutable": False,
                    "objective": False,
                }
            )

with open(OUTPUT, "w") as f:
    json.dump(result, f)